In [1]:
import tensorflow as tf

2024-08-18 11:59:48.179661: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 11:59:48.179764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 11:59:48.312967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tf.config.list_physical_devices('GPU')
# !pip install neattext

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.0 MB/s eta 0:00:00


In [3]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input,Dropout,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD,Adam
from tqdm import tqdm

In [4]:
data=pd.read_csv('/kaggle/input/suicide-watch/Suicide_Detection.csv')
data.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [5]:
data['class'].value_counts()

class
suicide        116037
non-suicide    116037
Name: count, dtype: int64

In [6]:
data['class'].value_counts().index.values

array(['suicide', 'non-suicide'], dtype=object)

In [7]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [8]:
train_data['class'].value_counts().index.values

array(['suicide', 'non-suicide'], dtype=object)

# **Data Visualisation**

In [9]:
import matplotlib.pyplot as plt

# Get the class counts
class_counts = train_data['class'].value_counts()

# Create the bar chart
plt.figure(figsize=(10, 6))
class_counts.plot(kind='bar')

# Customize the plot
plt.title('Distribution of Classes in Training Data')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0)

# Add value labels on top of each bar
for i, v in enumerate(class_counts):
    plt.text(i, v, str(v), ha='center', va='bottom')

# Show the plot
plt.tight_layout()
plt.show()

# **Data Cleaning**

In [10]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [11]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████| 46415/46415 [00:03<00:00, 13642.87it/s]


In [12]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [14]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=100)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=100)

In [15]:
train_text_pad

array([[    0,     0,     0, ...,   176,  3027,     3],
       [    0,     0,     0, ...,   163,   508,  1642],
       [    0,     0,     0, ...,    77,   240,    96],
       ...,
       [    0,     0,     0, ...,   328,     2,     4],
       [    0,     0,     0, ...,    65,    26,    16],
       [16008,  1861,     4, ...,     2,     4,    16]], dtype=int32)

# **Glove Embeddings**

In [16]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['class'])
test_output=lbl_target.transform(test_data['class'])

In [17]:
import pickle
# try with different glove models
with open('/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

# import numpy as np

# def load_glove_model(file_path):
#     print("Loading Glove Model")
#     glove_model = {}
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             split_line = line.split()
#             word = split_line[0]
#             embedding = np.array(split_line[1:], dtype=np.float64)
#             glove_model[word] = embedding
#     print(f"{len(glove_model)} words loaded!")
#     return glove_model

# # Load the GloVe model
# glove_file = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'  # Replace with your GloVe file path
# glove_embedding = load_glove_model(glove_file)

# Create the embedding matrix
v = len(tokenizer.word_index) + 1
embedding_dim = 300  # Dimension of GloVe vectors, adjust if using a different dimension

embedding_matrix = np.zeros((v+1, embedding_dim))
for word, idx in tokenizer.word_index.items():
    embedding_vector = glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector

In [18]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.074482  ,  0.58293003, -0.78233999, ..., -0.24984001,
        -0.096953  ,  0.66692001],
       [-0.35394999,  0.23051   , -0.62689   , ..., -0.20720001,
         0.52003002,  0.51129001],
       ...,
       [ 0.29547   , -0.21822999, -0.039817  , ...,  0.62642998,
         0.48798001, -0.47554001],
       [ 0.75085002, -0.35099   ,  0.37674999, ..., -0.066863  ,
         0.79632998, -0.05967   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
# early_stop=EarlyStopping(patience=5)
# reducelr=ReduceLROnPlateau(patience=3)

# **Keras Sequential Model Construction**

In [20]:
# model=Sequential()
# model.add(Input(shape=(50,)))
# # change the weight shape as per the dimensions
# model.add(Embedding(v+1,50,weights=[embedding_matrix],trainable=False))
# model.add(LSTM(units=64, input_shape=(50,), unroll=True, use_bias=True, recurrent_activation='sigmoid', return_sequences=True))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1,activation='sigmoid'))
# optimizer = Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# # model.compile(optimizer=SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reducelr = ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-5)

model = Sequential([
    Input(shape=(300,)),
    Embedding(v+1, 300, weights=[embedding_matrix], trainable=False),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'AUC'])


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 300, 300)       │    81,561,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 300, 128)       │       186,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,772,713 (311.94 MB)

 Trainable params: 211,713 (827.00 KB)

 Non-trainable params: 81,561,000 (311.13 MB)

# **Model Training and Evaluation**

In [23]:
r = model.fit(train_text_pad, train_output,
              validation_data=(test_text_pad, test_output),
              epochs=50,
              batch_size=1024,
              callbacks=[early_stop, reducelr])

Epoch 1/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - AUC: 0.9164 - accuracy: 0.8341 - loss: 0.3523 - val_AUC: 0.9778 - val_accuracy: 0.9272 - val_loss: 0.1892 - learning_rate: 0.0010
Epoch 2/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - AUC: 0.9786 - accuracy: 0.9308 - loss: 0.1835 - val_AUC: 0.9824 - val_accuracy: 0.9353 - val_loss: 0.1698 - learning_rate: 0.0010
Epoch 3/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - AUC: 0.9847 - accuracy: 0.9424 - loss: 0.1535 - val_AUC: 0.9847 - val_accuracy: 0.9340 - val_loss: 0.1748 - learning_rate: 0.0010
Epoch 4/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - AUC: 0.9891 - accuracy: 0.9532 - loss: 0.1286 - val_AUC: 0.9861 - val_accuracy: 0.9443 - val_loss: 0.1498 - learning_rate: 0.0010
Epoch 5/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 67ms/step - AUC: 0.9921 - accuracy: 0.9603 - loss: 0.1084 - val_AUC: 0.9866 - val_accuracy: 0.9457 - val_loss: 0.1478 - learning_rate: 0.0010
Epoch 6/50
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - AUC: 0.9940 - 

In [24]:
# r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
#             epochs=35,batch_size=2048,callbacks=[early_stop,reducelr])
# increasing the batch size is showing less increase in accuracy in each epochs but significantly less times so increase the epochs

In [25]:
y_pred = (model.predict(test_text_pad) > 0.5).astype("int32")
print('TESTING DATA CLASSIFICATION REPORT \n \n')
print(classification_report(test_output, y_pred,
                            target_names=lbl_target.inverse_transform([0,1])))

print('TRAINING DATA CLASSIFICATION REPORT \n \n')
y_pred_train = (model.predict(train_text_pad) > 0.5).astype("int32")
print(classification_report(train_output, y_pred_train,
                           target_names=lbl_target.inverse_transform([0,1])))

1451/1451 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
TESTING DATA CLASSIFICATION REPORT 
 

              precision    recall  f1-score   support

 non-suicide       0.94      0.95      0.95     23209
     suicide       0.95      0.94      0.95     23206

    accuracy                           0.95     46415
   macro avg       0.95      0.95      0.95     46415
weighted avg       0.95      0.95      0.95     46415

TRAINING DATA CLASSIFICATION REPORT 
 

5802/5802 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step
              precision    recall  f1-score   support

 non-suicide       0.98      0.96      0.97     92828
     suicide       0.96      0.98      0.97     92831

    accuracy                           0.97    185659
   macro avg       0.97      0.97      0.97    185659
weighted avg       0.97      0.97      0.97    185659



In [26]:
import pickle

In [27]:
twt = ['i am happy']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=100)

prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
else:
    print("Non Suicide Post")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.37726602
Non Suicide Post


In [28]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [29]:
model.save("model_840b_300d.h5")

In [30]:
token_form = pickle.load(open('tokenizer.pkl', 'rb'))

In [31]:
from tensorflow.keras.models import load_model

In [32]:
model_form = load_model("/kaggle/working/model_840b_300d.h5")

In [33]:

# twt = ["Through these past years thoughts of suicide, fear, anxiety I'm so close to my limit"]
# twt = ["I quit"]
twt = ["I'm so tired of everything, I don't see the point anymore."]
twt = token_form.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=100)


prediction = model_form.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
elif (prediction == 1):
    print("Non Suicide Post")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
0.9706215
Potential Suicide Post
